# TRAFFIC SIGNS CLASSIFICATION USING LE-NET ARCHITECTURE IN KERAS

## IMPORT LIBRARIES AND DATASET

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Import libraries
import pickle
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
# Load data
with open("./traffic-signs-data/train.p", mode='rb') as training_data:
    train = pickle.load(training_data)
with open("./traffic-signs-data/valid.p", mode='rb') as validation_data:
    valid = pickle.load(validation_data)
with open("./traffic-signs-data/test.p", mode='rb') as testing_data:
    test = pickle.load(testing_data)

In [ ]:
X_train, y_train = train['features'], train['labels']
X_validation, y_validation = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

In [ ]:
# Shuffle the dataset
X_train, y_train = shuffle(X_train, y_train)

### Single Image View

In [ ]:
i = int(input())
plt.imshow(X_train[i]) # Show images are not shuffled
y_train[i]

### DATA PEPARATION

In [ ]:
# Convert to grayscale
X_train_gray = np.sum(X_train / 3, axis=3, keepdims=True)
X_test_gray = np.sum(X_test / 3, axis=3, keepdims=True)
X_validation_gray = np.sum(X_validation / 3, axis=3, keepdims=True)

In [ ]:
# Normalize the images
X_train_gray_norm = (X_train_gray - 128) / 128
X_test_gray_norm = (X_test_gray - 128) / 128
X_validation_gray_norm = (X_validation_gray - 128) / 128

In [ ]:
X_train_gray.shape

In [ ]:
i = int(input())
plt.imshow(X_train_gray[i].squeeze(), cmap='gray')
plt.figure()
plt.imshow(X_train[i])

### MODEL TRAINING
The model consists of the following layers:

STEP 1: THE FIRST CONVOLUTIONAL LAYER #1

Input = 32x32x1
Output = 28x28x6
Output = (Input-filter+1)/Stride* => (32-5+1)/1=28
Used a 5x5 Filter with input depth of 3 and output depth of 6
Apply a RELU Activation function to the output
pooling for input, Input = 28x28x6 and Output = 14x14x6
Stride is the amount by which the kernel is shifted when the kernel is passed over the image.


STEP 2: THE SECOND CONVOLUTIONAL LAYER #2

Input = 14x14x6
Output = 10x10x16
Layer 2: Convolutional layer with Output = 10x10x16
Output = (Input-filter+1)/strides => 10 = 14-5+1/1
Apply a RELU Activation function to the output
Pooling with Input = 10x10x16 and Output = 5x5x16


STEP 3: FLATTENING THE NETWORK

Flatten the network with Input = 5x5x16 and Output = 400


STEP 4: FULLY CONNECTED LAYER

Layer 3: Fully Connected layer with Input = 400 and Output = 120
Apply a RELU Activation function to the output


STEP 5: ANOTHER FULLY CONNECTED LAYER

Layer 4: Fully Connected Layer with Input = 120 and Output = 84
Apply a RELU Activation function to the output


STEP 6: FULLY CONNECTED LAYER

Layer 5: Fully Connected layer with Input = 84 and Output = 43

In [ ]:
image_shape = X_train_gray[i].shape
image_shape

In [ ]:
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_validation_one_hot = to_categorical(y_validation, num_classes=num_classes)

In [ ]:
# Define LeNet model
cnn_model = Sequential()

cnn_model.add(Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 1)))
cnn_model.add(AveragePooling2D())

cnn_model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
cnn_model.add(AveragePooling2D())

cnn_model.add(Flatten())

cnn_model.add(Dense(units=120, activation='relu'))

cnn_model.add(Dense(units=84, activation='relu'))

cnn_model.add(Dense(units=43, activation='softmax'))

In [ ]:
# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Now you can train the model
cnn_model.fit(X_train_gray_norm,
              y_train,
              batch_size=500,
              epochs=5,
              verbose=1,
              validation_data=(X_validation_gray_norm, y_validation))

In [ ]:
cnn_model.summary()

In [ ]:
# Train the model
history = cnn_model.fit(X_train_gray_norm, y_train, batch_size=500, epochs=50, verbose=1,
                        validation_data=(X_validation_gray_norm, y_validation))

### MODEL EVALUATION

In [ ]:
# Evaluate the model
score = cnn_model.evaluate(X_test_gray_norm, y_test, verbose=0)
print('Test Accuracy: {:.4f}'.format(score[1]))

In [ ]:
history.history.keys()

In [ ]:
# Plot training history
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, accuracy, 'bo', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'ro', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Confusion matrix
predicted_classes = cnn_model.predict(X_test_gray_norm)
cm = confusion_matrix(y_test, predicted_classes)

In [ ]:
plt.figure(figsize=(15, 15))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
L = 10
W = 10
fig, axes = plt.subplots(L, W, figsize = (12,12))
axes = axes.ravel() # 

for i in np.arange(0, L * W):  
    axes[i].imshow(X_test[i])
#     axes[i].set_title("Prediction={}\n True={}".format(predicted_classes[i], y_true[i]))
    axes[i].axis('off')

plt.subplots_adjust(wspace=1)